In [23]:
import http.client
import json
import os
from xml_writer import XMLWriter
import urllib.request
import numpy as np
import cv2

base_url = '3.122.251.13'
username = 'auto-annotator-3000'
pwd  = 'beets-bears-battlestar-galactica'

if not os.path.exists('dataset_charts'):
        os.makedirs('dataset_charts')
if not os.path.exists('dataset_charts/Annotations'):
        os.makedirs('dataset_charts/Annotations')
if not os.path.exists('dataset_charts/ImageSets'):
        os.makedirs('dataset_charts/ImageSets')
if not os.path.exists('dataset_charts/JPEGImages'):
        os.makedirs('dataset_charts/JPEGImages')
        
if not os.path.exists('dataset_tables'):
        os.makedirs('dataset_tabless')
if not os.path.exists('dataset_tables/Annotations'):
        os.makedirs('dataset_tables/Annotations')
if not os.path.exists('dataset_tables/ImageSets'):
        os.makedirs('dataset_tables/ImageSets')
if not os.path.exists('dataset_tables/JPEGImages'):
        os.makedirs('dataset_tables/JPEGImages')
writer = XMLWriter()

In [24]:
connection = http.client.HTTPConnection(base_url)
body_dict = {'username':username, 'password':pwd}
connection.request('POST', '/api/users/login', json.dumps(body_dict), headers={'Content-Type':'application/json'})
response = connection.getresponse().read().decode()
token = json.loads(response)['token']

In [25]:
connection = http.client.HTTPConnection(base_url)
page_ids =  connection.request('GET', '/api/publications/pages?annotation_status=3:super_annotated', headers={'Authorization':'Token '+token})
response = connection.getresponse().read().decode()

In [26]:
data = json.loads(response)
results = data['results']

In [27]:
next_page = data['next']

In [30]:
table_ids = []
chart_ids = []

In [31]:
for result in results:
    page_id = result['id']
    print(page_id)
    image_url = result['image']
    connection = http.client.HTTPConnection(base_url)
    page_ids =  connection.request('GET', '/api/publications/annotations?page_id='+str(page_id), headers={'Authorization':'Token '+token})
    response = connection.getresponse().read().decode()
    data = json.loads(response)
    annotations = data['results']
    resp = urllib.request.urlopen(image_url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    img_width = image.shape[1]
    img_height = image.shape[0]
    plot_annotations = []
    table_annotations = []
    for annotation in annotations:
        if 'type' in annotation['data']:
            annotation_types = annotation['data']['type']
            if any(['plot' in x for x in annotation_types]):
                chart_ids.append(page_id)
                plot_annotations.append(annotation)
            if any(['table' in x for x in annotation_types]):
                table_ids.append(page_id)
                table_annotations.append(annotation)
    if any(plot_annotations):
        coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in plot_annotations]
        cv2.imwrite('dataset_charts/JPEGImages/'+str(page_id)+'.png', image)
        writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                              'dataset_charts',
                                              os.path.join(os.getcwd(), 'dataset_charts/JPEGImages', str(page_id)+'.png'),
                                              img_width,
                                              img_height,
                                              coords,
                                              'dataset_charts/Annotations/'+str(page_id)+'.xml')
    if any(table_annotations):
        coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in table_annotations]
        cv2.imwrite('dataset_tables/JPEGImages/'+str(page_id)+'.png', image)
        writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                              'dataset_tables',
                                              os.path.join(os.getcwd(), 'dataset_tables/JPEGImages', str(page_id)+'.png'),
                                              img_width,
                                              img_height,
                                              coords,
                                              'dataset_tables/Annotations/'+str(page_id)+'.xml')

5765
10957
11265
5882
5543
9370
328
6036
9871
702
1797
11093
3709
5043
2065
6367
6069
9979
11271
9228
9169
9282
5119
3573
966
4855
8752
109
8874
1531
8357
3600
536
4911
4919
8748
8457
2279
5027
781
8896
3996
617
4978
4327
828
1411
708
637
8220


In [ ]:
while next_page:
    print('page')
    connection = http.client.HTTPConnection(base_url)
    page_ids =  connection.request('GET', next_page[len('http://')+len(base_url):], headers={'Authorization':'Token '+token})
    response = connection.getresponse().read().decode()
    data = json.loads(response)
    results = data['results']
    next_page = data['next']
    for result in results:
        page_id = result['id']
        image_url = result['image']
        connection = http.client.HTTPConnection(base_url)
        page_ids =  connection.request('GET', '/api/publications/annotations?page_id='+str(page_id), headers={'Authorization':'Token '+token})
        response = connection.getresponse().read().decode()
        data = json.loads(response)
        annotations = data['results']
        resp = urllib.request.urlopen(image_url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        img_width = image.shape[1]
        img_height = image.shape[0]
        plot_annotations = []
        table_annotations = []
        for annotation in annotations:
            if 'type' in annotation['data']:
                annotation_types = annotation['data']['type']
                if any(['plot' in x for x in annotation_types]):
                    chart_ids.append(page_id)
                    plot_annotations.append(annotation)
                if any(['table' in x for x in annotation_types]):
                    table_ids.append(page_id)
                    table_annotations.append(annotation)
        if any(plot_annotations):
            coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in plot_annotations]
            cv2.imwrite('dataset_charts/JPEGImages/'+str(page_id)+'.png', image)
            writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                                  'dataset_charts',
                                                  os.path.join(os.getcwd(), 'dataset_charts/JPEGImages', str(page_id)+'.png'),
                                                  img_width,
                                                  img_height,
                                                  coords,
                                                  'dataset_charts/Annotations/'+str(page_id)+'.xml')
        if any(table_annotations):
            coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in table_annotations]
            cv2.imwrite('dataset_tables/JPEGImages/'+str(page_id)+'.png', image)
            writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                                  'dataset_tables',
                                                  os.path.join(os.getcwd(), 'dataset_tables/JPEGImages', str(page_id)+'.png'),
                                                  img_width,
                                                  img_height,
                                                  coords,
                                                  'dataset_tables/Annotations/'+str(page_id)+'.xml')

page


In [ ]:
connection = http.client.HTTPConnection(base_url)
page_ids =  connection.request('GET', '/api/publications/pages?annotation_status=2:annotated', headers={'Authorization':'Token '+token})
response = connection.getresponse().read().decode()
data = json.loads(response)
results = data['results']
next_page = data['next']
for result in results:
    page_id = result['id']
    print(page_id)
    image_url = result['image']
    connection = http.client.HTTPConnection(base_url)
    page_ids =  connection.request('GET', '/api/publications/annotations?page_id='+str(page_id), headers={'Authorization':'Token '+token})
    response = connection.getresponse().read().decode()
    data = json.loads(response)
    annotations = data['results']
    resp = urllib.request.urlopen(image_url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    img_width = image.shape[1]
    img_height = image.shape[0]
    plot_annotations = []
    table_annotations = []
    for annotation in annotations:
        if 'type' in annotation['data']:
            annotation_types = annotation['data']['type']
            if any(['plot' in x for x in annotation_types]):
                chart_ids.append(page_id)
                plot_annotations.append(annotation)
            if any(['table' in x for x in annotation_types]):
                table_ids.append(page_id)
                table_annotations.append(annotation)
    if any(plot_annotations):
        coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in plot_annotations]
        cv2.imwrite('dataset_charts/JPEGImages/'+str(page_id)+'.png', image)
        writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                              'dataset_charts',
                                              os.path.join(os.getcwd(), 'dataset_charts/JPEGImages', str(page_id)+'.png'),
                                              img_width,
                                              img_height,
                                              coords,
                                              'dataset_charts/Annotations/'+str(page_id)+'.xml')
    if any(table_annotations):
        coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in table_annotations]
        cv2.imwrite('dataset_tables/JPEGImages/'+str(page_id)+'.png', image)
        writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                              'dataset_tables',
                                              os.path.join(os.getcwd(), 'dataset_tables/JPEGImages', str(page_id)+'.png'),
                                              img_width,
                                              img_height,
                                              coords,
                                              'dataset_tables/Annotations/'+str(page_id)+'.xml')

In [ ]:
while next_page:
    print('page')
    connection = http.client.HTTPConnection(base_url)
    page_ids =  connection.request('GET', next_page[len('http://')+len(base_url):], headers={'Authorization':'Token '+token})
    response = connection.getresponse().read().decode()
    data = json.loads(response)
    results = data['results']
    next_page = data['next']
    for result in results:
        page_id = result['id']
        image_url = result['image']
        connection = http.client.HTTPConnection(base_url)
        page_ids =  connection.request('GET', '/api/publications/annotations?page_id='+str(page_id), headers={'Authorization':'Token '+token})
        response = connection.getresponse().read().decode()
        data = json.loads(response)
        annotations = data['results']
        resp = urllib.request.urlopen(image_url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        img_width = image.shape[1]
        img_height = image.shape[0]
        plot_annotations = []
        table_annotations = []
        for annotation in annotations:
            if 'type' in annotation['data']:
                annotation_types = annotation['data']['type']
                if any(['plot' in x for x in annotation_types]):
                    chart_ids.append(page_id)
                    plot_annotations.append(annotation)
                if any(['table' in x for x in annotation_types]):
                    table_ids.append(page_id)
                    table_annotations.append(annotation)
        if any(plot_annotations):
            coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in plot_annotations]
            cv2.imwrite('dataset_charts/JPEGImages/'+str(page_id)+'.png', image)
            writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                                  'dataset_charts',
                                                  os.path.join(os.getcwd(), 'dataset_charts/JPEGImages', str(page_id)+'.png'),
                                                  img_width,
                                                  img_height,
                                                  coords,
                                                  'dataset_charts/Annotations/'+str(page_id)+'.xml')
        if any(table_annotations):
            coords = [(int(x['data']['x1']*img_width), int(x['data']['x2']*img_width), int(x['data']['y1']*img_height), int(x['data']['y2']*img_height)) for x in table_annotations]
            cv2.imwrite('dataset_tables/JPEGImages/'+str(page_id)+'.png', image)
            writer.create_faster_rcnn_xml_multiple(str(page_id)+'.png',
                                                  'dataset_tables',
                                                  os.path.join(os.getcwd(), 'dataset_tables/JPEGImages', str(page_id)+'.png'),
                                                  img_width,
                                                  img_height,
                                                  coords,
                                                  'dataset_tables/Annotations/'+str(page_id)+'.xml')